### Import Excel Files To SQL

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
import urllib

# ---------- SQL Server connection details ----------
server = 'AG-SERVER-043'         # e.g. 'localhost\SQLEXPRESS'
database = 'harsh_data'
username = 'data05'
password = 'sai@123'

# ---------- Create the SQLAlchemy engine ----------
params = urllib.parse.quote_plus(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};PWD={password}"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# ---------- Folder path to start looking for Excel files ----------
#root_folder = r"E:\Back-Up Files\Data Team\Position & M2M\Intraday files"
root_folder = r"E:\Back-Up Files\Data Team\BSE - NSE\Bhavcopy_CM(MERGED)\JAN MERGE"
# ---------- Loop through folders and process Excel files ----------
for dirpath, _, filenames in os.walk(root_folder):
    for file in filenames:
        if file.lower().endswith('.xlsx'):
            file_path = os.path.join(dirpath, file)
            table_name = os.path.splitext(file)[0].replace(' ', '_').replace('-', '_')

            print(f"\n📥 Importing: {file_path} → Table: {table_name}")

            try:
                # Read the first sheet of the Excel file
                df = pd.read_excel(file_path)

                # Upload to SQL Server (replace table if exists)
                df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
                print(f"✅ Successfully imported into table: {table_name}")
            except Exception as e:
                print(f"❌ Failed to import {file}: {e}")

### Export SQL Files To Excel

In [7]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
import urllib

# ---------- SQL Server connection details ----------
server = 'AG-SERVER-043'
database = 'harsh_data'
username = 'data05'
password = 'sai@123'

# ---------- Output folder for Excel files ----------
output_folder = r"E:\Back-Up Files\Data Team\Position & M2M\NET FILES 2025"
os.makedirs(output_folder, exist_ok=True)

# ---------- Create the SQLAlchemy engine ----------
params = urllib.parse.quote_plus(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};PWD={password}"
)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# ---------- Define table name pattern ----------
prefix = 'EXP_ALLOVER_June_'
suffix = '_NET_'

# ---------- Fetch table names ----------
with engine.connect() as connection:
    result = connection.execute(text("""
        SELECT TABLE_NAME 
        FROM INFORMATION_SCHEMA.TABLES 
        WHERE TABLE_TYPE = 'BASE TABLE'
    """))
    all_tables = [row[0] for row in result]

# ---------- Filter tables matching the pattern ----------
april_tables = [tbl for tbl in all_tables if tbl.startswith(prefix) and tbl.endswith(suffix)]

# ---------- Export each matched table to Excel ----------
for table_name in april_tables:
    try:
        print(f"\n📤 Exporting table: {table_name}")
        df = pd.read_sql(f"SELECT * FROM [{table_name}]", con=engine)
        
        file_path = os.path.join(output_folder, f"{table_name}.xlsx")
        df.to_excel(file_path, index=False)
        print(f"✅ Exported to: {file_path}")
    except Exception as e:
        print(f"❌ Failed to export {table_name}: {e}")


📤 Exporting table: EXP_ALLOVER_June_2_NET_
✅ Exported to: E:\Back-Up Files\Data Team\Position & M2M\NET FILES 2025\EXP_ALLOVER_June_2_NET_.xlsx

📤 Exporting table: EXP_ALLOVER_June_3_NET_
✅ Exported to: E:\Back-Up Files\Data Team\Position & M2M\NET FILES 2025\EXP_ALLOVER_June_3_NET_.xlsx

📤 Exporting table: EXP_ALLOVER_June_4_NET_
✅ Exported to: E:\Back-Up Files\Data Team\Position & M2M\NET FILES 2025\EXP_ALLOVER_June_4_NET_.xlsx


### CSV import

In [2]:
import os
import pandas as pd
from sqlalchemy import create_engine
import urllib

# ---------- SQL Server connection details ----------
server = 'AG-SERVER-043'
database = 'harsh_data'
username = 'data05'
password = 'sai@123'

# ---------- Create the SQLAlchemy engine ----------
params = urllib.parse.quote_plus(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};PWD={password}"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# ---------- Folder path to start looking for CSV files ----------
root_folder = r"E:\Back-Up Files\Data Team\BSE - NSE\Bhavcopy_CM(MERGED)\FEB_2025_CM - Copy"

# ---------- Loop through folders and process CSV files ----------
for dirpath, _, filenames in os.walk(root_folder):
    for file in filenames:
        if file.lower().endswith('.csv'):
            file_path = os.path.join(dirpath, file)
            table_name = os.path.splitext(file)[0].replace(' ', '_').replace('-', '_')

            print(f"\n📥 Importing: {file_path} → Table: {table_name}")

            try:
                # Read the CSV file
                df = pd.read_csv(file_path, low_memory=False)


                # Upload to SQL Server (replace table if it exists)
                df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
                print(f"✅ Successfully imported into table: {table_name}")
            except Exception as e:
                print(f"❌ Failed to import {file}: {e}")



📥 Importing: E:\Back-Up Files\Data Team\BSE - NSE\Bhavcopy_CM(MERGED)\JAN MERGE\BHAVCOPY_2025_01_22_CM.csv → Table: BHAVCOPY_2025_01_22_CM
✅ Successfully imported into table: BHAVCOPY_2025_01_22_CM

📥 Importing: E:\Back-Up Files\Data Team\BSE - NSE\Bhavcopy_CM(MERGED)\JAN MERGE\BHAVCOPY_2025_01_23_CM.csv → Table: BHAVCOPY_2025_01_23_CM
✅ Successfully imported into table: BHAVCOPY_2025_01_23_CM

📥 Importing: E:\Back-Up Files\Data Team\BSE - NSE\Bhavcopy_CM(MERGED)\JAN MERGE\BHAVCOPY_2025_01_24_CM.csv → Table: BHAVCOPY_2025_01_24_CM
✅ Successfully imported into table: BHAVCOPY_2025_01_24_CM

📥 Importing: E:\Back-Up Files\Data Team\BSE - NSE\Bhavcopy_CM(MERGED)\JAN MERGE\BHAVCOPY_2025_01_27_CM.csv → Table: BHAVCOPY_2025_01_27_CM
✅ Successfully imported into table: BHAVCOPY_2025_01_27_CM

📥 Importing: E:\Back-Up Files\Data Team\BSE - NSE\Bhavcopy_CM(MERGED)\JAN MERGE\BHAVCOPY_2025_01_28_CM.csv → Table: BHAVCOPY_2025_01_28_CM
✅ Successfully imported into table: BHAVCOPY_2025_01_28_CM

📥 Im

#### BHAVCOPY IMPORT

In [3]:
import os
import pandas as pd
from sqlalchemy import create_engine, inspect
import urllib

# ---------- SQL Server connection details ----------
server = 'AG-SERVER-043'
database = 'harsh_data'
username = 'data05'
password = 'sai@123'

# ---------- Create the SQLAlchemy engine ----------
params = urllib.parse.quote_plus(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};PWD={password}"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# ---------- Folder path to start looking for CSV files ----------
root_folder = r"E:\Back-Up Files\Data Team\BSE - NSE\Bhavcopy_CM(MERGED)\Uoload_CM"

# ---------- Create an inspector to check existing tables ----------
inspector = inspect(engine)

# ---------- Loop through folders and process CSV files ----------
for dirpath, _, filenames in os.walk(root_folder):
    for file in filenames:
        if file.lower().endswith('.csv'):
            file_path = os.path.join(dirpath, file)
            table_name = os.path.splitext(file)[0].replace(' ', '_').replace('-', '_')

            # Check if table already exists
            if table_name in inspector.get_table_names():
                print(f"⏭️ Skipping {file} → Table '{table_name}' already exists.")
                continue

            print(f"\n📥 Importing: {file_path} → Table: {table_name}")

            try:
                # Read the CSV file
                df = pd.read_csv(file_path, low_memory = False)

                # Upload to SQL Server (fail if table already exists)
                df.to_sql(name=table_name, con=engine, if_exists='fail', index=False)
                print(f"✅ Successfully imported into table: {table_name}")
            except Exception as e:
                print(f"❌ Failed to import {file}: {e}")



📥 Importing: E:\Back-Up Files\Data Team\BSE - NSE\Bhavcopy_CM(MERGED)\Uoload_CM\BHAVCOPY_2025_01_23_CM.csv → Table: BHAVCOPY_2025_01_23_CM
✅ Successfully imported into table: BHAVCOPY_2025_01_23_CM

📥 Importing: E:\Back-Up Files\Data Team\BSE - NSE\Bhavcopy_CM(MERGED)\Uoload_CM\BHAVCOPY_2025_01_24_CM.csv → Table: BHAVCOPY_2025_01_24_CM
✅ Successfully imported into table: BHAVCOPY_2025_01_24_CM

📥 Importing: E:\Back-Up Files\Data Team\BSE - NSE\Bhavcopy_CM(MERGED)\Uoload_CM\BHAVCOPY_2025_01_27_CM.csv → Table: BHAVCOPY_2025_01_27_CM
✅ Successfully imported into table: BHAVCOPY_2025_01_27_CM

📥 Importing: E:\Back-Up Files\Data Team\BSE - NSE\Bhavcopy_CM(MERGED)\Uoload_CM\BHAVCOPY_2025_01_28_CM.csv → Table: BHAVCOPY_2025_01_28_CM
✅ Successfully imported into table: BHAVCOPY_2025_01_28_CM
